In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

np.random.seed(0)

In [2]:
data = pd.read_csv('C:/Users/Oksen/recsys/lesson2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

#### Перенесем нужные функции из вебинара

In [4]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [5]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [6]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [7]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = item_weights['item_id']
    weights = item_weights['weight']
    recs = np.random.choice(items, size=n, replace=False, p = weights)

    return recs.tolist()


In [8]:
item_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights['weight'] = item_weights['sales_value']/np.sum(item_weights['sales_value'])
item_weights.head(2)

,item_id,sales_value,weight
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07


In [9]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


#### Сделаем предсказания с помощью бейзлайн - алгоритмов без выделения топ-5000

In [10]:
items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items))


In [11]:
result['popularity_recommendation'] = result['user_id'].apply(lambda x: popularity_recommendation(data_train))


In [12]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights))
result.head(2)

,user_id,actual,random_recommendation,popularity_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13133794, 5579543, 12384762, 7409462, 1042390]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5567755, 835098, 1037337, 5572738, 998557]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1026922, 12385594, 850687, 9420068, 12356449]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1099372, 6534178, 865751, 873627, 1070272]"


In [16]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [17]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [18]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\Oksen\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


#### Попробуем улучшить бейзлайны, считая их на топ-5000 товаров

In [19]:
items = data_train.item_id.unique()
result['random_recommendation_top5000'] = result['user_id'].apply(lambda x: random_recommendation(items))

In [20]:
result['popularity_recommendation_top5000'] = result['user_id'].apply(lambda x: popularity_recommendation(data_train))

In [21]:
item_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights['weight'] = item_weights['sales_value']/np.sum(item_weights['sales_value'])
result['weighted_random_recommendation_top5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights))
result.head(2)

,user_id,actual,random_recommendation,popularity_recommendation,weighted_random_recommendation,random_recommendation_top5000,popularity_recommendation_top5000,weighted_random_recommendation_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13133794, 5579543, 12384762, 7409462, 1042390]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5567755, 835098, 1037337, 5572738, 998557]","[963502, 1099089, 9526886, 857390, 883912]","[999999, 6534178, 6533889, 1029743, 6534166]","[999999, 898068, 1001333, 872137, 838186]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1026922, 12385594, 850687, 9420068, 12356449]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1099372, 6534178, 865751, 873627, 1070272]","[931777, 1132231, 1084597, 937845, 9913338]","[999999, 6534178, 6533889, 1029743, 6534166]","[987218, 862349, 999999, 819643, 6534178]"


#### Рассчитаем метрики бейзлайн-алгоритмов, рассчитанные с применением топ-5000 и без.(Задание 2)

In [22]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

0.000979431929480901

In [23]:
result.apply(lambda row: precision_at_k(row['random_recommendation_top5000'], row['actual']), axis=1).mean()

0.006366307541625852

In [24]:
result.apply(lambda row: precision_at_k(row['popularity_recommendation'], row['actual']), axis=1).mean()

0.15523996082272082

In [25]:
result.apply(lambda row: precision_at_k(row['popularity_recommendation_top5000'], row['actual']), axis=1).mean()

0.08560235063663048

In [26]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()

0.02174338883447604

In [27]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation_top5000'], row['actual']), axis=1).mean()

0.02997061704211569

#### Попробуем разные варианты ItemItemRecommender, выбирая число соседей  𝐾 .

In [28]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [30]:
best_k, best_prec = 0,0

for i in range(1,6):

    model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

    result['itemitem' + str(i)] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])
    prec_at_k = result.apply(lambda row: precision_at_k(row['itemitem' + str(i)], row['actual']), axis=1).mean()
    if prec_at_k > best_prec:
        best_k = i
        best_prec = prec_at_k
    print(f'При к = {i} precision_at_k = {prec_at_k}');
    
print(f"Лучшее значение метрики {best_prec} достигается для K = {best_k}")    

При к = 1 precision_at_k = 0.1799869409076044
При к = 2 precision_at_k = 0.1920094678419821
При к = 3 precision_at_k = 0.1860920666013684
При к = 4 precision_at_k = 0.14495592556317158
При к = 5 precision_at_k = 0.13692458374142857
Лучшее значение метрики 0.1920094678419821 достигается для K = 2


### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.